## AdaBoost Classifier

In [1]:
import pandas as pd
import plotly.express as plt
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt


In [2]:
df =pd.read_csv("Travel.csv")
df

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,3.0,Deluxe,3.0,Single,1.0,1,2,1,0.0,Manager,20993.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,3,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Basic,3.0,Single,7.0,1,3,0,0.0,Executive,17090.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,2,3.0,Basic,3.0,Divorced,2.0,1,5,1,1.0,Executive,17909.0
4,200004,0,NaN,Self Enquiry,1,8.0,Small Business,Male,2,3.0,Basic,4.0,Divorced,1.0,0,5,1,0.0,Executive,18468.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4883,204883,1,49.0,Self Enquiry,3,9.0,Small Business,Male,3,5.0,Deluxe,4.0,Unmarried,2.0,1,1,1,1.0,Manager,26576.0
4884,204884,1,28.0,Company Invited,1,31.0,Salaried,Male,4,5.0,Basic,3.0,Single,3.0,1,3,1,2.0,Executive,21212.0
4885,204885,1,52.0,Self Enquiry,3,17.0,Salaried,Female,4,4.0,Standard,4.0,Married,7.0,0,1,1,3.0,Senior Manager,31820.0
4886,204886,1,19.0,Self Enquiry,3,16.0,Small Business,Male,3,4.0,Basic,3.0,Single,3.0,0,5,0,2.0,Executive,20289.0


### Data Cleaning

In [3]:
## chekcing null values
df.isnull().sum()

CustomerID                    0
ProdTaken                     0
Age                         226
TypeofContact                25
CityTier                      0
DurationOfPitch             251
Occupation                    0
Gender                        0
NumberOfPersonVisiting        0
NumberOfFollowups            45
ProductPitched                0
PreferredPropertyStar        26
MaritalStatus                 0
NumberOfTrips               140
Passport                      0
PitchSatisfactionScore        0
OwnCar                        0
NumberOfChildrenVisiting     66
Designation                   0
MonthlyIncome               233
dtype: int64

In [4]:
import numpy as np
ind =0
for i in df.columns:
    if type(df.loc[:, i])==object:
        print(i)


    ind +=1

In [5]:
df.loc[:, "MonthlyIncome"]

0       20993.0
1       20130.0
2       17090.0
3       17909.0
4       18468.0
         ...   
4883    26576.0
4884    21212.0
4885    31820.0
4886    20289.0
4887    24041.0
Name: MonthlyIncome, Length: 4888, dtype: float64

In [6]:
df["Gender"].value_counts()

Gender
Male       2916
Female     1817
Fe Male     155
Name: count, dtype: int64

In [7]:
df["Gender"].replace("Fe Male", "Female", inplace =True)

C:\Users\faiqi\AppData\Local\Temp\ipykernel_29384\1375468140.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Gender"].replace("Fe Male", "Female", inplace =True)


In [8]:
df["Gender"].value_counts()

Gender
Male      2916
Female    1972
Name: count, dtype: int64

In [9]:
df["MaritalStatus"].value_counts()

MaritalStatus
Married      2340
Divorced      950
Single        916
Unmarried     682
Name: count, dtype: int64

In [10]:
df["MaritalStatus"].replace("Single", "Unmarried", inplace=True)

C:\Users\faiqi\AppData\Local\Temp\ipykernel_29384\3994745670.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["MaritalStatus"].replace("Single", "Unmarried", inplace=True)


In [11]:
df["MaritalStatus"].value_counts()

MaritalStatus
Married      2340
Unmarried    1598
Divorced      950
Name: count, dtype: int64

### Imputing Null Values

In [12]:
# checking cols with null values
features_with_na =[f for f in df.columns if df[f].isnull().sum()>=1]
features_with_na

['Age',
 'TypeofContact',
 'DurationOfPitch',
 'NumberOfFollowups',
 'PreferredPropertyStar',
 'NumberOfTrips',
 'NumberOfChildrenVisiting',
 'MonthlyIncome']

In [13]:
df

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,3.0,Deluxe,3.0,Unmarried,1.0,1,2,1,0.0,Manager,20993.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,3,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Basic,3.0,Unmarried,7.0,1,3,0,0.0,Executive,17090.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,2,3.0,Basic,3.0,Divorced,2.0,1,5,1,1.0,Executive,17909.0
4,200004,0,NaN,Self Enquiry,1,8.0,Small Business,Male,2,3.0,Basic,4.0,Divorced,1.0,0,5,1,0.0,Executive,18468.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4883,204883,1,49.0,Self Enquiry,3,9.0,Small Business,Male,3,5.0,Deluxe,4.0,Unmarried,2.0,1,1,1,1.0,Manager,26576.0
4884,204884,1,28.0,Company Invited,1,31.0,Salaried,Male,4,5.0,Basic,3.0,Unmarried,3.0,1,3,1,2.0,Executive,21212.0
4885,204885,1,52.0,Self Enquiry,3,17.0,Salaried,Female,4,4.0,Standard,4.0,Married,7.0,0,1,1,3.0,Senior Manager,31820.0
4886,204886,1,19.0,Self Enquiry,3,16.0,Small Business,Male,3,4.0,Basic,3.0,Unmarried,3.0,0,5,0,2.0,Executive,20289.0


In [14]:


#Age
df.Age.fillna(df.Age.median(), inplace=True)

#TypeofContract
df.TypeofContact.fillna(df.TypeofContact.mode()[0], inplace=True)

#DurationOfPitch
df.DurationOfPitch.fillna(df.DurationOfPitch.median(), inplace=True)

#NumberOfFollowups
df.NumberOfFollowups.fillna(df.NumberOfFollowups.mode()[0], inplace=True)

#PreferredPropertyStar
df.PreferredPropertyStar.fillna(df.PreferredPropertyStar.mode()[0], inplace=True)

#NumberOfTrips
df.NumberOfTrips.fillna(df.NumberOfTrips.median(), inplace=True)

#NumberOfChildrenVisiting
df.NumberOfChildrenVisiting.fillna(df.NumberOfChildrenVisiting.mode()[0], inplace=True)

#MonthlyIncome
df.MonthlyIncome.fillna(df.MonthlyIncome.median(), inplace=True)




C:\Users\faiqi\AppData\Local\Temp\ipykernel_29384\700678725.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.Age.fillna(df.Age.median(), inplace=True)
C:\Users\faiqi\AppData\Local\Temp\ipykernel_29384\700678725.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

In [15]:
df.drop("CustomerID", axis=1, inplace=True)

## Feature Engineering

In [16]:
# create new column for feature
df['TotalVisiting'] = df['NumberOfPersonVisiting'] + df['NumberOfChildrenVisiting']
df.drop(columns=['NumberOfPersonVisiting', 'NumberOfChildrenVisiting'], axis=1, inplace=True)

### Train Test Split

In [17]:
X =df.drop("ProdTaken", axis=1)
Y =df["ProdTaken"]

In [18]:
Y.value_counts()

ProdTaken
0    3968
1     920
Name: count, dtype: int64

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test =train_test_split(X, Y, test_size=0.2, random_state=42)

## Column Transformer

In [20]:
cat =X.select_dtypes(include ="object").columns
num =X.select_dtypes(exclude="object").columns
# no need standard scaler in this case actually
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
standscaler =StandardScaler()
onehotencoder =OneHotEncoder(drop ="first")
preprocessor =ColumnTransformer([
    ("OneHotEncoder", onehotencoder, cat),
    ("StandardScaler", standscaler, num)
])


In [21]:
# applying on training data
X_train =preprocessor.fit_transform(X_train)
X_train

array([[ 1.        ,  0.        ,  0.        , ...,  0.78296635,
        -0.38224537, -0.77415132],
       [ 1.        ,  0.        ,  1.        , ...,  0.78296635,
        -0.4597992 ,  0.64361526],
       [ 1.        ,  1.        ,  0.        , ...,  0.78296635,
        -0.24519557, -0.06526803],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.78296635,
        -0.36057591,  0.64361526],
       [ 1.        ,  0.        ,  0.        , ...,  0.78296635,
        -0.25279888,  0.64361526],
       [ 0.        ,  0.        ,  1.        , ..., -1.2771941 ,
        -1.08251091, -1.48303461]])

In [22]:
# applying on test data
X_test =preprocessor.transform(X_test)
X_test

array([[ 0.        ,  0.        ,  0.        , ..., -1.2771941 ,
        -0.73751038, -0.77415132],
       [ 1.        ,  0.        ,  0.        , ..., -1.2771941 ,
        -0.6704111 , -0.06526803],
       [ 1.        ,  0.        ,  0.        , ...,  0.78296635,
        -0.4208322 , -0.77415132],
       ...,
       [ 0.        ,  1.        ,  0.        , ...,  0.78296635,
         0.69001249,  0.64361526],
       [ 1.        ,  0.        ,  0.        , ...,  0.78296635,
        -0.22827818, -0.77415132],
       [ 1.        ,  1.        ,  0.        , ...,  0.78296635,
        -0.44611323,  2.06138184]])

## Training Random Forest Classifier Model

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

In [25]:
models ={"Random Forest": RandomForestClassifier(), "Decison Tree":DecisionTreeClassifier(), "AdaBoost": AdaBoostClassifier() }
for i in range(len(list(models))):
    model =list(models.values())[i]


    
    model.fit(X_train, Y_train)


    # making pred
    y_pred_train =model.predict(X_train)
    y_pred_test  =model.predict(X_test)


    print(list(models.keys())[i])
    print("Model Performance for Trainning Data")
    print("Accuracy: ", accuracy_score(Y_train, y_pred_train))
    print("F1 Score: ", f1_score(Y_train, y_pred_train))
    print("Precision Score", precision_score(Y_train, y_pred_train))
    print("Recall Score", recall_score(Y_train, y_pred_train))
    print("Roc Auc Score: ", roc_auc_score(Y_train, y_pred_train))

    print("Model Performance for Testing Data")
    print("Accuracy: ", accuracy_score(Y_test, y_pred_test))
    print("F1 Score: ", f1_score(Y_test, y_pred_test))
    print("Precision Score", precision_score(Y_test, y_pred_test))
    print("Recall Score", recall_score(Y_test, y_pred_test))
    print("Roc Auc Score: ", roc_auc_score(Y_test, y_pred_test))
    print("-"*50)




Random Forest
Model Performance for Trainning Data
Accuracy:  1.0
F1 Score:  1.0
Precision Score 1.0
Recall Score 1.0
Roc Auc Score:  1.0
Model Performance for Testing Data
Accuracy:  0.9284253578732107
F1 Score:  0.78125
Precision Score 0.9689922480620154
Recall Score 0.6544502617801047
Roc Auc Score:  0.8246838348290613
--------------------------------------------------
Decison Tree
Model Performance for Trainning Data
Accuracy:  1.0
F1 Score:  1.0
Precision Score 1.0
Recall Score 1.0
Roc Auc Score:  1.0
Model Performance for Testing Data
Accuracy:  0.9202453987730062
F1 Score:  0.7925531914893617
Precision Score 0.8054054054054054
Recall Score 0.7801047120418848
Roc Auc Score:  0.8671806914720225
--------------------------------------------------


C:\Users\faiqi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost
Model Performance for Trainning Data
Accuracy:  0.8565217391304348
F1 Score:  0.4867337602927722
Precision Score 0.7307692307692307
Recall Score 0.36488340192043894
Roc Auc Score:  0.6670377399416719
Model Performance for Testing Data
Accuracy:  0.8353783231083844
F1 Score:  0.43109540636042404
Precision Score 0.6630434782608695
Recall Score 0.3193717277486911
Roc Auc Score:  0.6399908194016645
--------------------------------------------------


## HyperParameter Tunning

In [29]:
params ={"max_depth": [5, 8, 15, None, 10],
         "max_features": [5, 7, 10, 8] ,
         "min_samples_split": [2, 8, 15, 20],
         "n_estimators": [100, 200, 500, 1000]}
ada_params ={
    "n_estimators": [50,60,70,80,90],
    "algorithm": ["SAMME", "SAMME.R"]
}

In [30]:
# model list for hyperparamter tunning
modelcv =[("RF", RandomForestClassifier(), params), ("AB", AdaBoostClassifier(), ada_params)]

In [31]:
from sklearn.model_selection import RandomizedSearchCV
bestparams ={}
for name, model, para in modelcv:
    rd = RandomizedSearchCV(estimator=model, param_distributions=para, n_iter=100, cv =3, verbose=2, n_jobs=-1)
    rd.fit(X_train, Y_train)
    bestparams[name] = rd.best_params_

for model, bestparames in bestparams.items():
    print(model)
    print(bestparames)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


C:\Users\faiqi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 10 is smaller than n_iter=100. Running 10 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 10 candidates, totalling 30 fits
RF
{'n_estimators': 1000, 'min_samples_split': 2, 'max_features': 10, 'max_depth': None}
AB
{'n_estimators': 80, 'algorithm': 'SAMME'}


In [33]:
# training model with hyperparameters
models ={"Random Forest": RandomForestClassifier(n_estimators = 1000, min_samples_split = 2, max_features =10, max_depth = None), "AdaBoost": AdaBoostClassifier(n_estimators =80, algorithm ="SAMME")}
for i in range(len(list(models))):
    
    model =list(models.values())[i]


    
    model.fit(X_train, Y_train)


    # making pred
    y_pred_train =model.predict(X_train)
    y_pred_test  =model.predict(X_test)


    print("Model Performance for Testing Data")
    print("Accuracy: ", accuracy_score(Y_test, y_pred_test))
    print("F1 Score: ", f1_score(Y_test, y_pred_test))
    print("Precision Score", precision_score(Y_test, y_pred_test))
    print("Recall Score", recall_score(Y_test, y_pred_test))
    print("Roc Auc Score: ", roc_auc_score(Y_test, y_pred_test))


Model Performance for Testing Data
Accuracy:  0.9386503067484663
F1 Score:  0.8192771084337349
Precision Score 0.9645390070921985
Recall Score 0.7120418848167539
Roc Auc Score:  0.852844322332138
Model Performance for Testing Data
Accuracy:  0.83640081799591
F1 Score:  0.34959349593495936
Precision Score 0.7818181818181819
Recall Score 0.225130890052356
Roc Auc Score:  0.6049415568432047
